In [1]:
import nrrd
import numpy as np
import pandas as pd

In [2]:
folder_path = '../segmentations/'

In [3]:
file_name = 'Lung segmentation-AI-10.seg.nrrd'

In [4]:
data, header = nrrd.read( folder_path + file_name )

In [5]:
right_lung = data[0]
left_lung = data[1]

In [6]:
masks = {}
for i in range(data.shape[0]):
    seg_id = header.get(f"Segment{i}_ID")
    label_val = int(header.get(f"Segment{i}_LabelValue"))
    mask = data[i] == label_val
    masks[seg_id] = mask

In [7]:
print(masks)

{'right lung': array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, 

### Marching Cubes

In [14]:
from skimage import measure
from scipy.ndimage import binary_fill_holes
import pyvista as pv

In [16]:
# Volume data
right_data = masks['left lung']
lungs_data = masks['lungs']
filled_cavity = binary_fill_holes(right_data)

volume_data = lungs_data & filled_cavity
volume_data = np.nan_to_num(volume_data)

# Isosurface value
iso_value = 0.05

# Extract the isosurface
vertices, faces, normals, values = measure.marching_cubes(volume_data, iso_value)

ValueError: Surface level must be within volume data range.

In [11]:
def faces_to_edges(faces):
    edges = []
    for face in faces:
        edges.append(sorted([face[0],face[1]]))
        edges.append(sorted([face[1],face[2]]))
        edges.append(sorted([face[2],face[0]]))
    seen = set()
    unique_pairs = []
    
    for pair in edges:
        tup = tuple(pair)  # convert to tuple so it’s hashable
        if tup not in seen:
            seen.add(tup)
            unique_pairs.append(pair)
    return unique_pairs

In [12]:
edges = faces_to_edges(faces)

In [13]:
# Adjust faces array for PyVista
faces_pv = np.column_stack([np.full(len(faces), 3), faces])

# Create a PyVista mesh
mesh = pv.PolyData(vertices, faces_pv)

# Visualize the mesh
mesh.plot()

Widget(value='<iframe src="http://localhost:41031/index.html?ui=P_0x70bc0468eb70_0&reconnect=auto" class="pyvi…

In [ ]:
edges = mesh.extract_all_edges()
lines = edges.lines.reshape((-1, 3))

In [ ]:
export
for edge in lines:
    if edge[0] != 2:
        print('not 2')

In [ ]:
print(lines)

In [ ]:
print(vertices)